This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [24]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [25]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "train_1"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "test_1"    # e.g. './muffin-vs-chihuahua/test'

In [26]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [27]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.


Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [28]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    # layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    # layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    # layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

c:\Users\msi msi\OneDrive\Documents\Python\Artificial Neural Networks\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

c:\Users\msi msi\OneDrive\Documents\Python\Artificial Neural Networks\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 35s 287ms/step - accuracy: 0.7115 - loss: 0.5503 - val_accuracy: 0.8222 - val_loss: 0.4201
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 284ms/step - accuracy: 0.8268 - loss: 0.3992 - val_accuracy: 0.8561 - val_loss: 0.3504
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 283ms/step - accuracy: 0.8390 - loss: 0.3557 - val_accuracy: 0.8148 - val_loss: 0.3936
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 33s 280ms/step - accuracy: 0.8574 - loss: 0.3219 - val_accuracy: 0.8984 - val_loss: 0.2510
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 283ms/step - accuracy: 0.8754 - loss: 0.2944 - val_accuracy: 0.9153 - val_loss: 0.2087
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 282ms/step - accuracy: 0.8897 - loss: 0.2703 - val_accuracy: 0.9238 - val_loss: 0.2092
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 33s 280ms/step - accuracy: 0.8841 - loss: 0.2804 - val_accuracy: 0.9175 - val_loss: 0.2179
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 283ms/step - accuracy: 0.8936 - loss: 0

In [31]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 124ms/step - accuracy: 0.8446 - loss: 0.3661
Test Accuracy: 0.8445945978164673


In [32]:
# SAVE THE MODEL
model.save('muffin_vs_chihuahua_cnn.h5')

In [33]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='muffin_vs_chihuahua_cnn.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [34]:
# Example usage:
predict_image("run_1/run_1.jpg")
predict_image("run_1/run_2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


Prediction: Muffin (confidence: 0.27)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Prediction: Chihuahua (confidence: 0.57)
